In [81]:
#Computação científica
import numpy as np

#análise de dados
import pandas as pd

#visualização
import matplotlib.pyplot as plt

#machine learning
import sklearn

# feature engineering
from sklearn.impute import SimpleImputer
from feature_engine.imputation import (
    AddMissingIndicator)
from feature_engine.transformation import YeoJohnsonTransformer
from feature_engine.encoding import  RareLabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from feature_engine.wrappers import SklearnTransformerWrapper


#Pipelines
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

In [82]:
sklearn.set_config(display='diagram')
sklearn.set_config(transform_output="pandas")

# Introdução

## Motivação


## Objetivos


# Download Dataset

In [83]:
# O dataset está disponível no seguinte URL:
#https://www.kaggle.com/competitions/porto-seguro-safe-driver-prediction/data

path='/home/rodolfo/Insync/rodolfopcruz2@gmail.com/Google Drive/Estudo/Python_Projects/datasets/porto-seguro-safe-driver-prediction/'
train=pd.read_csv(path+'train.csv')
test= pd.read_csv(path+'test.csv')

In [84]:
cat_features=[feature for feature in train.columns if 'cat' in feature]

bin_features=[feature for feature in train.columns if 'bin' in feature]

num_features=[feature for feature in train.columns if 'cat' not in feature 
                                                    and 'bin' not in feature and
                                                    feature!='target' and feature!='id']


In [85]:
y_train=train['target']
train  =train.drop(columns=['target','id'])
test   =test.drop(columns=["id"])

In [86]:
#Nos dados os missing values estão representados por -1
#Substituir -1 por np.nan para facilitar a identificação
train=train.replace(-1,np.nan)

# Pipelines

- Serão criadas duas pipelines de dados:

    - Em uma delas serão tratadas todas as colunas, a seleção das features será feita mais a frente;
    - Na outra a primeira etapa consistirá na remoção de algumas festures, de acordo com os resultados obtidos com a correlação.

## Pipeline para dados com todas as features

In [87]:
#Nos dados os missing values foram substituídos pelo valor -1
#Função para trocar -1 por np .nan

def impute_nan_missing_value(x):
    x=x.replace(-1,np.nan)
    return x

impute_nan_missing_value_transformer=FunctionTransformer(impute_nan_missing_value)

In [88]:
#Função para converter data type para object

def converter_object(x):
    x=x.astype('object')
    return x

converter_object_transformer=FunctionTransformer(converter_object)

### Features Binárias

Pipeline para features binárias:

1) Substituir -1 por np.nan;
2) Converter os valores para object;
3) Substituir valores ausentes pelo mais frequente.

In [89]:
#imputer para substituir missing values
imputer_binary=SimpleImputer(strategy='most_frequent')

In [90]:
pipeline_bin_features=Pipeline([('replace_-1_nan',impute_nan_missing_value_transformer),
                          ('convert_to_object',converter_object_transformer),
                          ('fill_missing_binary',imputer_binary)])

### Features Numéricas

Pipeline para features numéricas:

1) Os missing values estão representados por -1. Substituir por np.nan;
2) Para as colunas numéricas com muitos missing values será criada um nova coluna para identificar um valor ausente;
3) Os valores ausentes de todas as colunas numéricas serão substituídos pela média;
4) Corrigir a distribuição de algumas colunas;
5) Normalização dos dados das colunas.


In [91]:
missing_threshold=1/100

#numeric features com muitos mussing values
num_features_muitos_na=train[num_features].isna().mean()>missing_threshold
num_features_muitos_na=num_features_muitos_na[num_features_muitos_na].index.to_list()

#numeric features com poucos missing values
num_features_poucos_na=train[num_features].isna().mean()<missing_threshold
num_features_poucos_na=num_features_poucos_na[num_features_poucos_na].index.to_list()

In [93]:
#Criar nova coluna para indicar se existe valor ausente nas colunas numéricas com muitos na

missing_indicator=AddMissingIndicator(variables=num_features_muitos_na)
#A coluna criada tem o mesmo nome da original acrescido da terminação _na

In [75]:
#Substituir os valores ausentes pela média
#Será aplicado em todas as colunas numéricas

num_imputer=SimpleImputer(strategy='mean')


In [74]:
#Aplicar transformação de Yeo Jhonson as seguintes features

features_to_be_transformed=['ps_reg_03',
                            'ps_car_12',
                            'ps_car_13',
                            'ps_car_14',
                            'ps_car_15',
                            'ps_reg_02']

yeo_transformer=YeoJohnsonTransformer(variables=features_to_be_transformed)

In [76]:
# Scaling

scaler = StandardScaler()

In [97]:
pipeline_num_features=Pipeline([
            ('replace_-1_nan',impute_nan_missing_value_transformer),
            ('addin_missing_indicator',missing_indicator),
            ('imputer_mean',SklearnTransformerWrapper(transformer=num_imputer,variables=num_features)),
            ('ajustar_ditribuicao',yeo_transformer),
            ('standard_scaler',SklearnTransformerWrapper(transformer=scaler,variables=num_features))])


### Features Categóricas

In [98]:
cat_features

['ps_ind_02_cat',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_car_01_cat',
 'ps_car_02_cat',
 'ps_car_03_cat',
 'ps_car_04_cat',
 'ps_car_05_cat',
 'ps_car_06_cat',
 'ps_car_07_cat',
 'ps_car_08_cat',
 'ps_car_09_cat',
 'ps_car_10_cat',
 'ps_car_11_cat']

In [99]:
missing_threshold=1/100
#features categóricas com número de missing values superior ao thresold
cat_features_muitos_na=train[cat_features].isna().mean()>missing_threshold
cat_features_muitos_na=cat_features_muitos_na[cat_features_muitos_na].index.to_list()

#features categóricas com número de missing values inferior ao thresold
cat_features_poucos_na=train[cat_features].isna().mean()<missing_threshold
cat_features_poucos_na=cat_features_poucos_na[cat_features_poucos_na].index.to_list()

In [139]:
#Substituir missing values nas features com muitos valores ausentes
cat_imputer_muitos_na=SimpleImputer(strategy='constant',fill_value='missing')

#Substituir missing values nas features com poucos valores ausentes
cat_imputer_poucos_na=SimpleImputer(strategy='most_frequent')


In [140]:
#Labels raras

rare_threshold=1/100
#Todas as labels que aparecem em proproção inferior a rare_thrshold serão agrupadas como uma única

#Identificar fearues que contem labels raras
rare_labels=[]
for feature in cat_features:
    if  not ((train[feature].value_counts()/len(train))>rare_threshold).all():
        rare_labels.append(feature)

rare_encoder = RareLabelEncoder(tol=rare_threshold, n_categories=1, variables=rare_labels)


In [141]:
#One hot encoder
enc = OneHotEncoder(handle_unknown='ignore',sparse_output=False)

#one hot encoding será aplicado somente a features com mais de duas labels
#e tambem aquelas com duas labels mas com muitos missing values, para a quais foi criada um label para indicar um missing value
columns_to_encode=[x for x in cat_features if train[x].nunique()>2 or x in cat_features_muitos_na]


In [147]:
#Converter para string para que tods os dados em uma mesma coluna tenham o mesmo formato
#Essa etapa é necessária para usar o one hot encoding

def converter_para_str(x):
    x=x.astype(str)
    return x

converter_str_transformer=FunctionTransformer(converter_para_str)

In [148]:
pipeline_cat_features=Pipeline([
            ('replace_-1_nan',impute_nan_missing_value_transformer),
            ('converter_object',converter_object_transformer),
            ('imputer_muitos_na',SklearnTransformerWrapper(transformer=cat_imputer_muitos_na,variables=cat_features_muitos_na)),
            ('imputer_poucos_na',SklearnTransformerWrapper(transformer=cat_imputer_poucos_na,variables=cat_features_poucos_na)),
            ('rare_labels',rare_encoder),
            ('converter_str',SklearnTransformerWrapper(transformer=converter_str_transformer,variables=columns_to_encode)),
            ('one_hot',SklearnTransformerWrapper(transformer=enc,variables=columns_to_encode))])